Линейная регрессия — это один из основных и наиболее простых методов машинного обучения, используемый для предсказания непрерывных значений на основе одного или нескольких входных признаков. Основная идея линейной регрессии заключается в нахождении линейной зависимости между входными признаками и целевой переменной.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = None

df = pd.read_csv("houses.csv")

df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df = df.drop('House', axis=1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

x = df['SquareFootage'].values.reshape(-1, 1)
y = df['Price'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
plt.scatter(x, y)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_train, y_train)
y_pred = model.predict(x_test)

score = model.score(x, y)
print('model score:', score)

In [ ]:
x

In [ ]:
y

In [ ]:
x_test
y_pred

In [ ]:
plt.scatter(x, y)
plt.plot(x_test, y_pred, color = 'g')
plt.show()